In [3]:
import requests  
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import mysql.connector as SQLC
import mysql.connector
from datetime import date
import time
from bs4 import BeautifulSoup
from Bio.SeqFeature import CompoundLocation
from datetime import datetime

In [59]:
#Extrair ids genebank:

data_e_hora_atuais = datetime.now()

query= input('escolha o que quer pesquisar: ')


try:
    def url_get(i):
        url_list_id=[]
        url = f"https://www.ncbi.nlm.nih.gov/gene/?term={i}"
        url_list_id.append(url)
        return url_list_id
    
    url_get(query)

    content = []
    for url in url_get(query):
        r = requests.get(url)
        content.append(r.content)

    for c in content:
        soup = BeautifulSoup(c, 'html.parser')
        a= soup.get_text()

    existe = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

    c= ', '.join(existe)
    h= c.replace('ID: ','')
    IDS= h.split(', ')

    n_gene= IDS[0:1+(int(input('escolha o nº de genes que quer obter (máximo 20): ')))]

    n_genes = []
    seen = set()
    for item in n_gene:
        if item not in seen:
            seen.add(item)
            n_genes.append(item)

    numero_genes= len(n_genes)

    if n_genes == ['']:
        print()
        print('0 resultados para a sua pesquisa, pesquise de novo.')

except:
    ('0 resultados para a sua pesquisa, pesquise de novo.')

escolha o que quer pesquisar: gli2
escolha o nº de genes que quer obter (máximo 20): 20


In [60]:
#Extrair ids ncbi:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
    
if Ids == ['']:
    print('0 resultados para a sua pesquisa, pesquise de novo.')
        
#Extrair description NCBI:
description=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    description.append(info.description)

#organismos
Organismos=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Organismos.append(info.annotations['organism'])
    
#SEQ
seqss=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    seqs= (f'{info.seq[0:10]}...{info.seq[-10:]}')
    seqss.append(seqs)

#percentagem de nucle:
Adenina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    c=info.seq
    count= c.count('A')
    perc= int(count/len(c)*100)
    Adenina.append(perc)

Citosina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    c=info.seq
    count= c.count('C')
    perc= int(count/len(c)*100)
    Citosina.append(perc)

Guanina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    c=info.seq
    count= c.count('G')
    perc= int(count/len(c)*100)
    Guanina.append(perc)

Timina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    c=info.seq
    count= c.count('T')
    perc= int(count/len(c)*100)
    Timina.append(perc)
    
#data
dates=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    dates.append(info.annotations['date'])

#len(SEQ)
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
tam=[]
for info in records:
    tam.append(len(info.seq))
    
    # buscar info pubmed
try:
    def url_get_id(i):
        url_list= [ ]
        for id in i:
            url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
            url_list.append(url)
        return url_list
    link_genebank= url_get_id(Ids)
    #print(link_genebank)

    content_id = []
    for url in url_get_id(Ids):
        r_id = requests.get(url)
        content_id.append(r_id.content)

    from bs4 import BeautifulSoup
    listas=[]

    for c in content_id:
        soup_id = BeautifulSoup(c, 'html.parser')

        lines = soup_id.find_all('meta', {'name':"ncbi_uidlist"} )

        id = ""
        url = ""
        for line in lines:
            if 'content' in line.attrs:
                id = line.attrs['content']
        if id:
            url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id)
        r2 = requests.get(url)
        r4= r2.content.decode('utf-8')
        listas.append(r4)

    cc= ', '.join(listas)
    er= cc.replace('//','')
    final= er.split(', ')

    #criar dicionário de ids ncbi e ids pubmed
    output_dict = {}
    for x in final:
        version = re.search(r'VERSION\s+(.*?)\s', x)
        pubmed = re.search(r'PUBMED\s+(.*?)\s', x)
        if version:
            versionf=version.group(1)
            output_dict.setdefault(version.group(1), [])
        if pubmed:
            output_dict[versionf].append(pubmed.group(1))
except:
    print()
    print("0 resultados na Pubmed em realção à sua query")
    
#Criar Listas só com ids ncbi e ids pubmed
id_ncbii = []
ID_PUB = []
for key, vals in output_dict.items():
    if vals:
        for val in vals:
            id_ncbii.append(key)
            ID_PUB.append(val)
    else:
        id_ncbii.append(key)
        ID_PUB.append("N/A")

new_list_ = []
seen = set()
for item in ID_PUB:
    if item not in seen:
        seen.add(item)
        new_list_.append(item)

print("Concluido")

Concluido


In [61]:
#Extrair informação de artigos
try:
    titles=[]
    authors=[]
    source=[]
    affiliation=[]
    database = 'PubMed'
    email= 'rodrigoce9@gmail.com'
    idlist= new_list_
    counter = 0
    for i in idlist:
        if i!= "N/A":
            handle = Entrez.efetch(db=database, id=i, rettype="medline", retmode="text") 
            records = Medline.parse(handle)
            for info in records:
                titles.append(info.get("TI", ["N/A"]))
                authors_string = info.get("AU", ["N/A"])
                if len(authors_string) > 5:
                    authors_h = authors_string[0:5]
                    authors.append(authors_h)
                else:
                    authors.append(authors_string) 
                source.append(info.get("SO", ["N/A"]))
                affiliation_string= info.get("AD", ["N/A"])
                if len(affiliation_string) > 5:
                    affiliation_h = affiliation_string[0:5]
                    affiliation.append(affiliation_h)
                else:
                    affiliation.append(affiliation_string)
                counter += 1
        else:
            titles.append(["N/A"])
            authors.append(["N/A"])
            source.append(["N/A"])
            affiliation.append(["N/A"])
            counter += 1
except:
    print('artigos não encontrados por um possivel bug de id')
    
#agrupar titles
titles = [ [title] for title in titles]

# agrupar dois
doi_list = []
for x in source:
    match = re.search("doi: (.*)", str(x))
    if match:
        doi_list.append(match.group(1))
    else:
        doi_list.append("N/A")
        
#agrupar authors
id_authors_dict = {i: authors[counter] if i != "N/A" else ["N/A"] for counter, i in enumerate(idlist)}        
        
pubmed_list = []
authors_list = []
for key, vals in id_authors_dict.items():
    if vals:
        for val in vals:
            pubmed_list.append(key)
            authors_list.append(val)

new_list_authors= []
seen = set()
for item in authors_list:
    if item not in seen:
        seen.add(item)
        new_list_authors.append(item)

#agrupar affi
single_affiliation_list = []
for i in affiliation:
    single_affiliation_list.extend(i)
    

id_affiliation_dict = {i: [single_affiliation_list[counter]] if i != "N/A" else ["N/A"] for counter, i in enumerate(idlist)}

pubmed_affi_list = []
affi_list = []
for key, vals in id_affiliation_dict.items():
    if vals:
        for val in vals:
            pubmed_affi_list.append(key)
            affi_list.append(val)
    else:
        pubmed_affi_list.append(key)
        affi_list.append(["N/A"])

new_list_affi= []
seen = set()
for item in single_affiliation_list :
    if item not in seen:
        seen.add(item)
        new_list_affi.append(item)

def url_get_id_p(string):
    url_list_p=[ ]
    url_id_p= "https://www.uniprot.org/uniprotkb/{}/entry".format(str(string))
    url_list_p.append(url_id_p)
    return ''.join(url_list_p)

# protein_id - PROT_ID 
list_pro=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    for i in info.features:
        if i.type == "CDS":
            pro= str(i.qualifiers["protein_id"])
            list_pro.append(pro)
div= ', '.join(list_pro)
h= div.replace("['",'')
hh= h.replace("']",'')
ID_PROT= hh.split(', ')

#result_dic = {Ids, ID_PROT} - 
result_dict = {}
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist=Ids
for ids in idlist:
    list_pro=[]
#     print( ids)
    handle = Entrez.efetch(db=database, id=ids, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    for info in records:
        list_pro.append(info.id)
        for i in info.features:
            if i.type == "CDS":
                pro= str(i.qualifiers["protein_id"])
                list_pro.append(pro)
        if len(list_pro)==1: #if no protein_id was found
            result_dict[info.id] = "N/A_CDS"
        else:
            list_pro = [x.replace("['",'').replace("']",'') for x in list_pro]
            result_dict[info.id] = ', '.join(list_pro[1:])
#key: id_genebank; values: ID_PROT

def dict_to_list(d, delimiter=','): #transforms dictionary to list
    new_list = []
    for k, v in d.items():
        if isinstance(v, str):
            values = v.split(delimiter)
            for val in values:
                new_list.append([k, val.strip()])
        else:
            new_list.append([k, v])
    return new_list
dict_list=dict_to_list(result_dict)


from collections import OrderedDict
def get_CDS_info(result_dict): #gets info for CDS [ID_CDS, location, Translation]
    database = 'nucleotide'
    email= 'rodrigoce9@gmail.com'
    Entrez.email = email
    cds_location_list = []
    several_location = []
    record_types={}
    processed_i = set()
    for i, value in result_dict.items():
        if i in processed_i:
            continue
        if value == 'N/A_CDS':
            cds_location = ['N/A_CDS', 'N/A', 'N/A']
            cds_location_list.append(cds_location)
        else:
            handle = Entrez.efetch(db=database, id=i, rettype="gb") 
            records = list(SeqIO.parse(handle,"gb"))
            handle.close()
            for info in records:
                for i in info.features:
                    if i.type == "CDS":
                        cds_location = []
                        i_d = str(i.qualifiers["protein_id"])
                        translation =  str(i.qualifiers["translation"])
                        cds_location.append(i_d)
                        if isinstance(i.location, CompoundLocation):
                            for sub_location in i.location.parts:
                                several_location.append("[{} : {}]".format(sub_location.start, sub_location.end))
                            cds_location.append(several_location)
                            cds_location.append(translation)
                        else:
                            cds_location.append("[{} : {}]".format(i.location.start, i.location.end))
                            cds_location.append(translation)
                        cds_location_list.append(cds_location)      
                processed_i.add(i)
            handle.close()
    return cds_location_list   

WEBSITE_API = "https://rest.uniprot.org"
fields = ["accession","organism_name","protein_name","cc_subcellular_location","cc_function", "sequence"]
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);
    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()
    return response

#get list of [ID_Uniprot, Subcellular location, Function, Sequence, Sequence length]
def get_field_for_id(ID_PROT, field): 
    response = get_url("{}/uniprotkb/search?query={}&fields={}&size=1&format=tsv".format(WEBSITE_API,ID_PROT,field))
    return str(response.content)

try:
    def get_list_uniprot(ID_PROT, result_dict):
        results = []
        result = []
        tmp= []
        easy=dict_to_list(result_dict)
        for first_index, first_value in easy:
            tmp= []
            if first_value != 'N/A_CDS':
                for field in fields:
                    result = get_field_for_id(first_value, field)
                    tmp.append(result)
            else:
                result = ['N/A_Uniprot']
                tmp.append(result)
            results.append(tmp)   
        uniprot_final_list=[]
        for index, values in enumerate(results):
            uniprot_list=[]
            n_a = re.search(r'(N/A_Uniprot)', str(values), re.DOTALL)
            if len (values)==1:
                if n_a:
                    uniprot_list.append(n_a.group(1))
                    tmp_list='N/A','N/A','N/A','N/A'
                    uniprot_list.extend(tmp_list)
            else:
                for i in values:
                    entry = re.search(r'b\'Entry\\n(.+?(?=\\n\'))', str(i), re.DOTALL)
                    function = re.match( r'b\'Function \[CC\]\\n.{9} (.+?(?=\\n\'))', str(i), re.DOTALL )
                    location_exist = re.search( r'b\'Subcellular location \[CC\]\\nSUBCELLULAR LOCATION: (.+?(?=\\n\'))', str(i), re.DOTALL )
                    location_notexist = re.search( r'b\'Subcellular location \[CC\]\\n\\n\'',str(i), re.DOTALL )   
                    sequence = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', str(i), re.DOTALL)
                    if entry:
                        ent=entry.group(1)
                        uniprot_list.append(entry.group(1))            
                    if function:
                        uniprot_list.append(function.group(1))
                    if location_exist:
                        uniprot_list.append(location_exist.group(1))
                    if location_notexist:
                        uniprot_list.append("N/A")
                    if sequence:
                        uniprot_list.append(sequence.group(1))
                        uniprot_list.append(len(sequence.group(1)))
                        uniprot_list.append(url_get_id_p(ent))

            if len(uniprot_list) < 6: #sometimes, there is no function associated
                uniprot_list.insert(2, 'N/A')
            uniprot_final_list.append(uniprot_list)

        return uniprot_final_list
    get_Uniprot=get_list_uniprot(ID_PROT,result_dict)    
except:
    print("A informação foi apagada ")
    
#[idgenebank, protein_id, id do uniprot] 
#joins to final list [idgenebank, ID_PRO, ID_Uniprot]
def join_ids_CDS(lista, uniprotID): 
    join_list_all = []
    for key, value in lista:
        join_list = []
        join_list.append(key)
        join_list.append(value)
        join_list_all.append(join_list) 
    for index, values in enumerate(uniprotID):
        join_list_all[index].append(uniprotID[index][0])
    return join_list_all
join_ids=join_ids_CDS(dict_list,get_list_uniprot(ID_PROT, result_dict))


def join_lists(list1, list2): #joins to final list [idgenebank, ID_PRO, ID_Uniprot,location, Translation]
    final_result =[]
    list_tmp=[]
    tmp=''
    for item, value in enumerate(list1):
        result = []
        string_item = str(list2[item][0])
        string_item = string_item.replace("[","").replace("]","").replace("'","")
        if list1[item][1] == string_item:
            result.append(list1[item] + list2[item][1:])
            tmp=list1[item][0]
        elif list1[item][0] == tmp:
            list_tmp = []
            list_tmp.append(tmp)
            list_tmp.append(list1[item][1])
            list_tmp.append(list1[item][2])
            result.append(list_tmp + list2[item][1:])
        else:
            result.append(list1[item])
        final_result.extend(result)
    return final_result
join_CDS=join_lists(join_ids, get_CDS_info(result_dict))

def count_(genes):
    return len(genes)
count_(Ids)
count_(ID_PROT)



print("Concluido")

Concluido


In [62]:
#delete de tudo 

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
        
        a= "delete from PubMed_Affiliation"
        Z= "delete from PubMed_Authors"
        b= "delete from Affiliation"
        c= "delete from Authors"
        d= "delete from gene_PubMed"
        e= "delete from PubMed"
        f= "delete from CDS"
        g= "delete from Uniprot"
        h= "delete from Gene"
        #i= "delete from History"     
        
        Cursor.execute(a)
        Cursor.execute(Z)
        Cursor.execute(b)
        Cursor.execute(c)
        Cursor.execute(d)
        Cursor.execute(e)
        Cursor.execute(f)
        Cursor.execute(g)
        Cursor.execute(h)
        #Cursor.execute(i)
        
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [63]:
#Povoação "History"

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql1= "INSERT INTO History (search, Genes_number_input, Day, Genes_number_NCBI, Protein_number  )   VALUES (%s, %s, %s, %s, %s)"
    val1=(query, numero_genes, data_e_hora_atuais, count_(Ids), count_(ID_PROT) )
    Cursor.execute(sql1,val1)
    
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) ) 
    
search_id=[]
try:
    sql2= "Select ID_search FROM History"
    Cursor.execute(sql2)
    for row in Cursor:
        search_id.append(str(row)) 
        
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
div= ', '.join(search_id)
h= div.replace("(",'')
hh= h.replace(",)",'')
SEARCH_ID= hh.split(', ')
Hist= SEARCH_ID[-1]

#Povoação "Gene"-

des = (description)

try:
    for index, value in enumerate(Ids):
       
        sql3= "INSERT INTO Gene (ID_genebank, Description, Organism, sequence, Date_publish, length, Adenina, Citosina, Guanina, Timina, Link, ID_search) VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val3=(value, des[index], Organismos[index], seqss[index], dates[index], tam[index], Adenina[index], Citosina[index], Guanina[index], Timina[index], link_genebank[index], Hist)

        Cursor.execute(sql3,val3)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
#Povoar "PubMed"

try:
    for index, value in enumerate(new_list_):
        sql4= "INSERT INTO PubMed (ID_PumMed, title, Doi_number) VALUES (%s, %s, %s)"
        val4=(str(new_list_[index]), str(titles[index]), str(doi_list[index]) )
        Cursor.execute(sql4,val4)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    

    
#retirar os valores AI de pubmed
ID_AI=[]

try:
    sql5= "Select ID_AI_PubMed FROM PubMed"
    Cursor.execute(sql5)
    for row in Cursor:
        ID_AI.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
      
div_= ', '.join(ID_AI)
h_= div_.replace("(",'')
hh_= h_.replace(",)",'')
SEARCH_ID_= hh_.split(', ')
#print(SEARCH_ID_)

#Povação "Gene-Pubmed"
number_map = {}
next_number = int(SEARCH_ID_[0])
relation = []
for number in ID_PUB:
    if number not in number_map:
        number_map[number] = next_number
        next_number += 1
    relation.append(number_map[number])
    
try:
    for index, value in enumerate(id_ncbii):
        
        sql6= "INSERT INTO gene_PubMed (ID_genebank, ID_AI_PubMed) VALUES (%s, %s)"
        val6=(id_ncbii[index], relation[index])
    
        Cursor.execute(sql6,val6)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
#Povação "authors"
try:
    for index, value in enumerate(new_list_authors):
        
        sql7= "INSERT INTO Authors (Name) VALUES (%s)"
        val7=(new_list_authors[index],)
    
        Cursor.execute(sql7,val7)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    

#retirar os valores AI de authors
ID_AI_Authors=[]

try:
    sql8= "Select ID_Authors FROM Authors"
    Cursor.execute(sql8)
    for row in Cursor:
        ID_AI_Authors.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
div_Authors= ', '.join(ID_AI_Authors)
h_Authors= div_Authors.replace("(",'')
hh_Authors= h_Authors.replace(",)",'')
SEARCH_ID_Authors= hh_Authors.split(', ')
#print(SEARCH_ID_Authors)

#Povação "Pubmed-Authors"
number_map_ = {}
next_number = int(SEARCH_ID_Authors[0])
output_authors = []
for number in authors_list:
    if number not in number_map_:
        number_map_[number] = next_number
        next_number += 1
    output_authors.append(number_map_[number])

number_map_pub = {}
next_number = int(SEARCH_ID_[0])
output_pub = []
for number in pubmed_list:
    if number not in number_map_pub:
        number_map_pub[number] = next_number
        next_number += 1
    output_pub.append(number_map_pub[number])

try:
    for index, value in enumerate(output_pub):
        
        sql9= "INSERT INTO PubMed_Authors (ID_AI_PubMed, ID_Authors) VALUES (%s, %s)"
        val9=(str(output_pub[index]),str(output_authors[index]))
    
        Cursor.execute(sql9,val9)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
#Povação "affi"

try:
    for index, value in enumerate(new_list_affi):
        
        sql10= "INSERT INTO Affiliation (Info) VALUES (%s)"
        val10=(new_list_affi[index],)
    
        Cursor.execute(sql10,val10)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
#retirar os valores AI de affi
ID_AI_Affiliation=[]

try:
    sql11= "Select ID_Affiliation FROM Affiliation"
    Cursor.execute(sql11)
    for row in Cursor:
        ID_AI_Affiliation.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
div_Affiliation= ', '.join(ID_AI_Affiliation)
h_Affiliation= div_Affiliation.replace("(",'')
hh_Affiliation= h_Affiliation.replace(",)",'')
SEARCH_ID_Affiliation= hh_Affiliation.split(', ')
#print(SEARCH_ID_Affiliation)

#Povação "Pubmed-Affiliation"
number_map_affi_pub = {}
next_number = int(SEARCH_ID_[0])
output_pub_affi = []
for number in pubmed_affi_list:
    if number not in number_map_affi_pub:
        number_map_affi_pub[number] = next_number
        next_number += 1
    output_pub_affi.append(number_map_affi_pub[number])
    
number_map_affi = {}
next_number = int(SEARCH_ID_Affiliation[0])
output_affi = []
for number in affi_list:
    if number not in number_map_affi:
        number_map_affi[number] = next_number
        next_number += 1
    output_affi.append(number_map_affi[number])

try:
    for index, value in enumerate(output_pub_affi):
        
        sql12= "INSERT INTO PubMed_Affiliation (ID_AI_PubMed, ID_Affiliation) VALUES (%s, %s)"
        val12=(str(output_pub_affi[index]),str(output_affi[index]))
    
        Cursor.execute(sql12,val12)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    

#Povoação "Uniprot" 
# print(get_Uniprot) #(ID_Uniprot, Subcelular location, Function, Protein seq, length_aa) 

a=["N/A_Uniprot"]

try:
    sec1 = 'INSERT INTO Uniprot (ID_Uniprot, Subcelular_Location, Function, Protein_sequence, length_aa, Link_Uniprot) VALUES (%s, %s, %s, %s, %s, %s)'
    Cursor.execute(sec1, (a[0],a[0],a[0],a[0],a[0],a[0]))
    for index, value in enumerate(get_Uniprot):
        if str(value[0]) == 'N/A_Uniprot':
            continue
        else:
            sql13= "INSERT INTO Uniprot (ID_Uniprot, Subcelular_Location, Function, Protein_sequence, length_aa, Link_Uniprot) VALUES (%s, %s, %s, %s, %s, %s)"
            val13=(str(value[0]), str(value[1]), str(value[2]), str(value[3]), str(value[4]), str(value[5]))
            Cursor.execute(sql13,val13)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    

#Povoação "CDS"
try:
    for item in join_CDS:
        sql14= "INSERT INTO CDS (ID_CDS, Translation, Location, ID_genebank, ID_Uniprot) VALUES (%s, %s, %s, %s, %s)"
        val14=(str(item[1]),str(item[4]),str(item[3]),str(item[0]),str(item[2]))
        Cursor.execute(sql14,val14)
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

print("Concluido")

Concluido


In [7]:
# import pandas as pd 

# DataBase = SQLC.connect(
#     host ="geo.di.uminho.pt",
#     user ="bioinformatica",
#     password ="20221207",
#     database ="AP_db_KRG"
# )
# tabela1 = pd.read_sql("Select * FROM History", DataBase)
# print(tabela1)
# tabela2 = pd.read_sql("Select * FROM Gene", DataBase)
# print(tabela2)
# tabela3 = pd.read_sql("Select * FROM gene_PubMed", DataBase)
# print(tabela3)
# tabela4 = pd.read_sql("Select * FROM PubMed", DataBase)
# print(tabela4)
# tabela5 = pd.read_sql("Select * FROM PubMed_Authors", DataBase)
# print(tabela5)
# tabela6 = pd.read_sql("Select * FROM Authors", DataBase)
# print(tabela6)
# tabela7 = pd.read_sql("Select * FROM PubMed", DataBase)
# print(tabela7)
# tabela8 = pd.read_sql("Select * FROM PubMed_Affiliation", DataBase)
# print(tabela8)
# tabela9 = pd.read_sql("Select * FROM Affiliation", DataBase)
# print(tabela9)
# tabela10 = pd.read_sql("Select * FROM CDS", DataBase)#print(tabela10)
# print(tabela10)
# tabela11 = pd.read_sql("Select * FROM Uniprot", DataBase)
# print(tabela11)
# DataBase.close()

# Algoritmos:

In [4]:
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True 
Cursor = DataBase.cursor()

num= input("Escolha o tamanho máximo das seq: ")
Seq_and_length=[]
just_seq=[]
try:
    sql_Seq_length= f"select Gene.ID_genebank, Gene.length from Gene where Gene.length <{num}"
    Cursor.execute(sql_Seq_length)
    for row in Cursor:
        Seq_and_length.append(str(row)) 
    
    sql_Seq= f"select Gene.ID_genebank from Gene where Gene.length <{num}"
    Cursor.execute(sql_Seq)
    for row in Cursor:
        just_seq.append(str(row))
        
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
    
print (Seq_and_length)

if just_seq == []:
        print()
        print('nenhuma seq inferior a esse tamanho, pesquise de novo.')


Escolha o tamanho máximo das seq: 50
["('AAADK0043071.1', 21)", "('AAADS0003348.1', 20)", "('AAADW0009410.1', 21)", "('AAADX0043451.1', 21)", "('ABAAM0346030.1', 20)"]


In [5]:
list_seq=[]

choose= input('escolha a primeira seq com que quer trabalhar: ')
inter= int(choose)
extr= ', '.join(just_seq)
h= extr.replace("(",'')
hh= h.replace(",)",'')
ID_al= hh.split(', ')
select= ID_al[inter]
list_seq.append(select)

choose2= input('escolha a segunda seq com que quer trabalhar: ')
inter2= int(choose2)
extr2= ', '.join(just_seq)
h2= extr2.replace("(",'')
hh2= h2.replace(",)",'')
ID_al2= hh2.split(', ')
select2= ID_al2[inter2]
list_seq.append(select2)

limpar_aspas= ', '.join(list_seq)
retirar= limpar_aspas.replace("'","")
otput_final= retirar.split(', ')
print(otput_final)

escolha a primeira seq com que quer trabalhar: 0
escolha a segunda seq com que quer trabalhar: 1
['AAADK0043071.1', 'AAADS0003348.1']


In [69]:
# #SEQ_algoritmos
# seq_for_al=[]
# database = 'nucleotide'
# email= 'rodrigoce9@gmail.com'
# idlist= otput_final
# for x in idlist:
#     handle = Entrez.efetch(db=database, id=x, rettype="gb") 
#     records = SeqIO.read(handle,"gb")
#     handle.close()
#     print(records.seq)
#     print()
#     seq_for_al.append(records.seq)

GAGTGGAGATGGCGGAGCTGT

CAACACGGGAAACCTCACCC



In [6]:
#SEQ_algoritmos
seq_for_al=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist1= otput_final[0]
idlist2= otput_final[1]
handle1 = Entrez.efetch(db=database, id=idlist1, rettype="gb") 
records1 = SeqIO.read(handle1,"gb")
handle1.close()
s1_=records1.seq
handle2 = Entrez.efetch(db=database, id=idlist2, rettype="gb") 
records2 = SeqIO.read(handle2,"gb")
handle2.close()
s2_=records2.seq

print(s1_)
print(s2_)


/Users/utilizador/opt/anaconda3/lib/python3.9/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


GAGTGGAGATGGCGGAGCTGT
CAACACGGGAAACCTCACCC


### Matriz:

In [7]:
import io

class Mat:
    
    def __init__(self, rows, cols):
        self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]
    def numRows (self): return len(self.mat)
    def numCols (self): return len(self.mat[0])
    def __str__(self):
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)
    def __repr__(self):
        return str(self)
    def __getitem__ (self, n):
        return self.mat[n]

In [407]:
#classe nw a funcionar
def subst(x, y):
    return 3 if x == y else -1 #match de 3 mismatch -1

class NW:
    def __init__(self, s1, s2, g = -4):
        self.s1 = s1 
        self.s2 = s2 
        self.mat = Mat(len(s1) + 1, len(s2) + 1)
        self.tr  = Mat(len(s1) + 1, len(s2) + 1)

        for L in range(len(s1)):
            self.mat[L + 1][0] = g * (L + 1) 
            self.tr[L + 1][0]  = 'C' 
        for C in range(len(s2)):
            self.mat[0][C + 1] = g * (C + 1) 
            self.tr[0][C + 1]  = 'E'

        for L, x1 in enumerate(s1):
            for C, x2 in enumerate(s2):
                possiveis = [
                    self.mat[L  ][C    ] + subst(x1, x2),   # Diagonal
                    self.mat[L+1][C    ] + g,               # Esquerda
                    self.mat[L  ][C + 1] + g,               # Cima
                ]
                dirs = "DEC"
                self.mat[L + 1][C + 1] = max(possiveis)
                self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]
                
    def rebuild(self):
        L = len(self.s1)
        C = len(self.s2)
        S1 = ""
        S2 = ""
    
        dirs = {
            'D' : (-1, -1),
            'E' : ( 0, -1),
            'C' : (-1,  0)
        }
        while L > 0 or C > 0:
            
            DL, DC = dirs[self.tr[L][C]]
            if self.tr[L][C] == "D":
                S1 = self.s1[L - 1] + S1 
                S2 = self.s2[C - 1] + S2
            elif self.tr[L][C] == "E":
                S1 = '-' + S1
                S2 = self.s2[C - 1] + S2
            else:
                S1 = self.s1[L - 1] + S1
                S2 = '-' + S2        
    
            L += DL
            C += DC

        return S1, S2 
    def __repr__(self): 
        cols = "-" + self.s2
        lins = "-" + self.s1
        with io.StringIO("") as S:
            print(' ', *cols, sep = '   ', file = S)  
            for L, linha in zip(lins, self.mat):
                print(L, *[f'{x:3d}' for x in linha], file = S)
            print(file = S)

            print(' ', *cols, file = S) 
            for L, linha in zip(lins, self.tr):
                print(L, *linha, file = S)

            return S.getvalue()
        
alin= NW(s1_,s2_, g= -4)
print (alin)
print()
print(*alin.rebuild(), sep="\n")


    -   C   A   A   C   A   C   G   G   G   A   A   A   C   C   T   C   A   C   C   C
-   0  -4  -8 -12 -16 -20 -24 -28 -32 -36 -40 -44 -48 -52 -56 -60 -64 -68 -72 -76 -80
G  -4  -1  -5  -9 -13 -17 -21 -21 -25 -29 -33 -37 -41 -45 -49 -53 -57 -61 -65 -69 -73
A  -8  -5   2  -2  -6 -10 -14 -18 -22 -26 -26 -30 -34 -38 -42 -46 -50 -54 -58 -62 -66
G -12  -9  -2   1  -3  -7 -11 -11 -15 -19 -23 -27 -31 -35 -39 -43 -47 -51 -55 -59 -63
T -16 -13  -6  -3   0  -4  -8 -12 -12 -16 -20 -24 -28 -32 -36 -36 -40 -44 -48 -52 -56
G -20 -17 -10  -7  -4  -1  -5  -5  -9  -9 -13 -17 -21 -25 -29 -33 -37 -41 -45 -49 -53
G -24 -21 -14 -11  -8  -5  -2  -2  -2  -6 -10 -14 -18 -22 -26 -30 -34 -38 -42 -46 -50
A -28 -25 -18 -11 -12  -5  -6  -3  -3  -3  -3  -7 -11 -15 -19 -23 -27 -31 -35 -39 -43
G -32 -29 -22 -15 -12  -9  -6  -3   0   0  -4  -4  -8 -12 -16 -20 -24 -28 -32 -36 -40
A -36 -33 -26 -19 -16  -9 -10  -7  -4  -1   3  -1  -1  -5  -9 -13 -17 -21 -25 -29 -33
T -40 -37 -30 -23 -20 -13 -10 -11  -8  -5  -1   2  -2 

In [409]:
class SW:
    def __init__(self, s1, s2, gap_penalty, match_score, mismatch_penalty):
        self.s1 = s1 
        self.s2 = s2 
        self.gap_penalty= gap_penalty
        self.match_score= match_score
        self.mismatch_penalty= mismatch_penalty
        self.score = [[0 for j in range(len(s2) + 1)] for i in range(len(s1) + 1)]
        self.tr = [[0 for j in range(len(s2) + 1)] for i in range(len(s1) + 1)]
       

        for L in range(len(s1)): 
            self.tr[L + 1][0]  = 'E' 
        for C in range(len(s2)):
            self.tr[0][C + 1]  = 'C'
                
        for i in range(1, len(self.s1) + 1):
            for j in range(1, len(self.s2) + 1):
                match = self.score[i - 1][j - 1] + (self.match_score if self.s1[i - 1] == self.s2[j - 1] else self.mismatch_penalty)
                delete = self.score[i - 1][j] + self.gap_penalty
                insert = self.score[i][j - 1] + self.gap_penalty
                self.score[i][j] = max(0, match, delete, insert)
                possiveis= (match, delete, insert,0)
                dirs = 'DEC0'
                self.tr[i][j] = dirs[possiveis.index(self.score[i][j])]
                
                
    def rebuild(self):
    
        L = len(self.s1)
        C = len(self.s2)
            
        max_i, max_j = 0, 0
        max_score = 0
        for i in range(1, L + 1):
            for j in range(1, C + 1):
                if self.score[i][j] > max_score:
                    max_i, max_j = i, j
                    max_score = self.score[i][j]
        i, j = max_i, max_j
        alignD1= "" 
        alignD2= ""
        while self.score[i][j] != 0:
            current_score = self.score[i][j]
            diagonal_score = self.score[i - 1][j - 1]
            up_score = self.score[i][j - 1]
            left_score = self.score[i - 1][j]

            if current_score == diagonal_score + (self.match_score if self.s1[i - 1] == self.s2[j - 1] else self.mismatch_penalty):
                alignD1 += self.s1[i - 1]
                alignD2 += self.s2[j - 1]
                i -= 1
                j -= 1
            elif current_score == left_score + self.gap_penalty:
                alignD1 += self.s1[i - 1]
                alignD2 += "-"
                i -= 1
            elif current_score == up_score + self.gap_penalty:
                alignD1 += "-"
                alignD2 += self.s2[j - 1]
                j -= 1
        
                
        return alignD1[::-1], alignD2[::-1] 
    
    
    
    def __repr__(self): 
        cols = "-" + self.s2
        lins = "-" + self.s1
        with io.StringIO("") as S:
            print(' ', *cols, sep = '   ', file = S)  
            for L, linha in zip(lins, self.score):
                print(L, *[f'{x:3d}' for x in linha], file = S)
            print(file = S)

            print(' ', *cols, file = S) 
            for L, linha in zip(lins, self.tr ):
                print(L, *linha, file = S)

            return S.getvalue()


alignSW= SW(s1_,s2_,-4,3,-1)
print(alignSW)
print(*alignSW.rebuild(), sep="\n")

    -   C   A   A   C   A   C   G   G   G   A   A   A   C   C   T   C   A   C   C   C
-   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
G   0   0   0   0   0   0   0   3   3   3   0   0   0   0   0   0   0   0   0   0   0
A   0   0   3   3   0   3   0   0   2   2   6   3   3   0   0   0   0   3   0   0   0
G   0   0   0   2   2   0   2   3   3   5   2   5   2   2   0   0   0   0   2   0   0
T   0   0   0   0   1   1   0   1   2   2   4   1   4   1   1   3   0   0   0   1   0
G   0   0   0   0   0   0   0   3   4   5   1   3   0   3   0   0   2   0   0   0   0
G   0   0   0   0   0   0   0   3   6   7   4   0   2   0   2   0   0   1   0   0   0
A   0   0   3   3   0   3   0   0   2   5  10   7   3   1   0   1   0   3   0   0   0
G   0   0   0   2   2   0   2   3   3   5   6   9   6   2   0   0   0   0   2   0   0
A   0   0   3   3   1   5   1   1   2   2   8   9  12   8   4   0   0   3   0   1   0
T   0   0   0   2   2   1   4   0   0   1   4   7   8 

### Multiple align:

In [410]:
s3='AATCGGTCA'
s4='GGTTGGGAC'

def consensus(s1_, s2_):
    res = ""
    for x, y in zip(s1_, s2_):
        if x == y:
            res += x
        elif x == '-':
            res += y
        else:
            res += x
    #print (res)
    return res

a1, a2 = NW(s1_, s2_, g = -1).rebuild()
print(a1)
print(a2)
print()
m1 = consensus(a1,a2)
print(m1)
print()
m2, a3 = NW(m1, s3, g = -1).rebuild()
print(m2,a3, sep="\n")
print()
m2= consensus(a2,a3)
print(m2)
print()
m3, a4= NW(m2,s4,g=-1).rebuild()
print(m3)
print(a4)
print()


---GA-GTGGAGA--TGGCGGAGCTGT
CAACACG-GGAAACCT--C--A-C-CC

CAAGACGTGGAGACCTGGCGGAGCTGT

CAAGACGTGGAGACCTGGCGGAGCTGT
--A-A--TCG-G---T--C--A-----

CAACACGTGGAAACCT--C--A-C-CC

CAACACG-TGGAAACCT--C--A-C-CC
-----GGTTGG----------GA----C

